In [4]:
import yfinance as yf
import pandas as pd
import numpy as np


In [5]:
# Step 1: Download close prices from Yahoo Finance
start_date = "2013-08-21"
end_date = "2023-08-21"
tickers = ["AAPL", "GOOGL", "META"]

data = yf.download(tickers, start=start_date, end=end_date)['Close']


[*********************100%%**********************]  3 of 3 completed


In [6]:
# Step 2: Merge dataframes on Date column
merged_data = pd.concat([data[ticker] for ticker in tickers], axis=1)
merged_data.columns = tickers

In [7]:
# Step 3: Calculate log returns of close price for each of the stock
log_returns = np.log(merged_data / merged_data.shift(1))

In [8]:
# Step 4: Calculate VaR for different confidence levels
confidence_levels = [0.99, 0.95, 0.9]
var_results = {}

for confidence in confidence_levels:
    var_results[confidence] = log_returns.quantile(1 - confidence)

In [9]:
# Step 5: Repeat for different holding periods
holding_periods = [1, 2, 5, 10]
var_results_by_period = {}

for period in holding_periods:
    var_results_by_period[period] = {}
    for confidence in confidence_levels:
        var_results_by_period[period][confidence] = var_results[confidence] * np.sqrt(period)


In [10]:
# Step 6: Create a summary dataframe
summary_data = pd.DataFrame(index=holding_periods, columns=['Confidence Level'] + confidence_levels)

for period, results in var_results_by_period.items():
    summary_data.at[period, 'Confidence Level'] = f"{period}day"
    for confidence, var_value in results.items():
        summary_data.at[period, confidence] = var_value

# Print the summary dataframe
print(summary_data)


   Confidence Level                                               0.99  \
1              1day  AAPL    -0.049210
GOOGL   -0.048157
META    -0...   
2              2day  AAPL    -0.069593
GOOGL   -0.068105
META    -0...   
5              5day  AAPL    -0.110037
GOOGL   -0.107683
META    -0...   
10            10day  AAPL    -0.155615
GOOGL   -0.152287
META    -0...   

                                                 0.95  \
1   AAPL    -0.027129
GOOGL   -0.026460
META    -0...   
2   AAPL    -0.038366
GOOGL   -0.037420
META    -0...   
5   AAPL    -0.060661
GOOGL   -0.059167
META    -0...   
10  AAPL    -0.085788
GOOGL   -0.083675
META    -0...   

                                                  0.9  
1   AAPL    -0.019024
GOOGL   -0.018838
META    -0...  
2   AAPL    -0.026905
GOOGL   -0.026641
META    -0...  
5   AAPL    -0.042540
GOOGL   -0.042123
META    -0...  
10  AAPL    -0.060160
GOOGL   -0.059570
META    -0...  
